# ReverseNumber Example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread

In [3]:
#USER_JWT_PATH = "/home/staverm/workspace/coflows-dev/coflows/jwts.txt"
#user_jwt = open(USER_JWT_PATH).readline().strip()
#cl = CoLink("http://127.0.0.1:2021", user_jwt)

In [4]:
cl = colink_utils.start_colink_server()

# Start a few default workers

In [5]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 143268b3-eb23-4e65-9df1-12810dea86c6
owner_id: local
message_paths: ['push_tasks:5a9ec746-2547-4eca-9381-3540a9c314c9:msg']
parallel_dispatch: False

Input message source: Proxy_reverse_number_sequential
Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 143268b3-eb23-4e65-9df1-12810dea86c6
owner_id: local
message_paths: ['push_tasks:b8e6efbf-6deb-473b-8984-10bd293bcf3d:msg']
parallel_dispatch: False

Input message source: ReverseNumberFirst
Called ReverseNumberSequential 
 state {'current_call': 'second_reverse_flow', 'initial_message': FlowMessage(message_id='44a92119-a68f-41c9-884c-ec144d71205b', created_at='2024-03-19 22:46:25.513067415', created_by='Proxy_reverse_number_sequential', message_type='FlowMessage', data={'id': 0, 'number': 

In [6]:
run_dispatch_worker_thread(cl)

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: eedc43bd-6154-42a0-90b0-7f276fb17aa1
owner_id: local
message_paths: ['push_tasks:2b651b32-02e2-495c-a0ef-3c08623915f4:msg']
parallel_dispatch: False

Input message source: first_reverse_flow
Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: eedc43bd-6154-42a0-90b0-7f276fb17aa1
owner_id: local
message_paths: ['push_tasks:422c6f7b-bf45-417e-bb0e-096234f8b2d7:msg']
parallel_dispatch: False

Input message source: first_reverse_flow
Called ReverseNumberAtomic 
 state {}


# Serve

In [7]:
serve_utils.recursive_serve_flow(
    cl=cl,
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential"
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True

Subflow second_reverse_flow already served.
Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False



True

## Get instance

In [8]:
flow = serve_utils.get_flow_instance(
    cl=cl,
    flow_endpoint="reverse_number_sequential",
    user_id="local",
)

Mounted eedc43bd-6154-42a0-90b0-7f276fb17aa1 at flows:reverse_number_atomic:mounts:local:eedc43bd-6154-42a0-90b0-7f276fb17aa1
Fetched singleton eedc43bd-6154-42a0-90b0-7f276fb17aa1
Mounted 143268b3-eb23-4e65-9df1-12810dea86c6 at flows:reverse_number_sequential:mounts:local:143268b3-eb23-4e65-9df1-12810dea86c6


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
reply_data = flow.get_reply_future(input_message).get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



## Observe memory

In [10]:
def status():
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)
status()

 reverse_number_sequential/
   init
   mounts/
     local/
       143268b3-eb23-4e65-9df1-12810dea86c6/
         state
         config_overrides
         init
   default_dispatch_point
   singleton
   parallel_dispatch
   flow_class_name
 reverse_number_atomic/
   singleton
   parallel_dispatch
   mounts/
     local/
       eedc43bd-6154-42a0-90b0-7f276fb17aa1/
         config_overrides
         state
         init
   flow_class_name
   init
   default_dispatch_point

All flow instances:
 143268b3-eb23-4e65-9df1-12810dea86c6
 eedc43bd-6154-42a0-90b0-7f276fb17aa1
